## **Importing Modules**

In [22]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Embedding, Dense, Flatten, LSTM, Bidirectional, BatchNormalization
from keras.callbacks import EarlyStopping
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [2]:
print("GPUs: ")
print(keras.backend.tensorflow_backend._get_available_gpus())

GPUs: 
['/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1']


## **Setting Parameters**

In [3]:
nb_epochs = 100
batch_size = 32

## **Loading Data**

40,000 lines of Shakespeare from a variety of Shakespeare's plays. 
Featured in Andrej Karpathy's blog post 'The Unreasonable Effectiveness of Recurrent Neural Networks': http://karpathy.github.io/2015/05/21/rnn-effectiveness/

In [4]:
with open("input.txt") as f:
    corpus = list(f)
# Removing line break characters
corpus = [sent[:-1] for sent in corpus]
# Removing empty strings
corpus = [sent for sent in corpus if sent]

## **Data Preprocessing**

In [5]:
# Tokenizing the data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)

In [6]:
# Total vocabulary size
vocab_size = len(tokenizer.word_index) + 1

In [7]:
# Converting text to input sequences
input_sequences = []
for sent in corpus:
    token_list = tokenizer.texts_to_sequences([sent])[0]
    for i in range(1, len(token_list)):
        input_sequences.append(token_list[:i+1])

In [8]:
# Example input sequences
input_sequences[:5]

[[88, 269],
 [139, 35],
 [139, 35, 969],
 [139, 35, 969, 143],
 [139, 35, 969, 143, 668]]

In [9]:
# Length of the longest sequence
max_seq_len = max([len(seq) for seq in input_sequences])

In [10]:
# Padding the sequences
padded_input_sequences = pad_sequences(input_sequences, maxlen=max_seq_len, padding = "pre")
# Splitting the sequences into features and labels
train_data = padded_input_sequences[:, :-1]
train_labels = padded_input_sequences[:, -1]

In [11]:
# One-Hot encoding the labels
train_labels = to_categorical(train_labels, num_classes=vocab_size)

## **Model Architecture**

In [23]:
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_seq_len-1))
model.add(Bidirectional(LSTM(100)))
model.add(BatchNormalization())
model.add(Dense(vocab_size, activation = "softmax"))

In [24]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 15, 100)           1263300   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 200)               160800    
_________________________________________________________________
batch_normalization_1 (Batch (None, 200)               800       
_________________________________________________________________
dense_5 (Dense)              (None, 12633)             2539233   
Total params: 3,964,133
Trainable params: 3,963,733
Non-trainable params: 400
_________________________________________________________________


In [25]:
model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [26]:
# Early stop callback
early_stop = EarlyStopping(monitor = "loss", patience = 8)

## **Fitting the Model**

In [ ]:
model.fit(train_data, train_labels, epochs=nb_epochs, batch_size=batch_size, callbacks = [early_stop])

Epoch 1/100
171312/171312 [==============================] - 212s 1ms/step - loss: 6.4321 - acc: 0.0859
Epoch 2/100
171312/171312 [==============================] - 210s 1ms/step - loss: 5.6903 - acc: 0.1177
Epoch 3/100
171312/171312 [==============================] - 211s 1ms/step - loss: 5.2180 - acc: 0.1429
Epoch 4/100
171312/171312 [==============================] - 211s 1ms/step - loss: 4.7588 - acc: 0.1752
Epoch 5/100
171312/171312 [==============================] - 211s 1ms/step - loss: 4.3394 - acc: 0.2118
Epoch 6/100
171312/171312 [==============================] - 209s 1ms/step - loss: 3.9863 - acc: 0.2519
Epoch 7/100
171312/171312 [==============================] - 210s 1ms/step - loss: 3.7109 - acc: 0.2857
Epoch 8/100
171312/171312 [==============================] - 215s 1ms/step - loss: 3.4990 - acc: 0.3150
Epoch 9/100
171312/171312 [==============================] - 210s 1ms/step - loss: 3.3267 - acc: 0.3411
Epoch 10/100
171312/171312 [==============================] - 20

171312/171312 [==============================] - 206s 1ms/step - loss: 1.4475 - acc: 0.6963
Epoch 99/100
171312/171312 [==============================] - 207s 1ms/step - loss: 1.4430 - acc: 0.6957
Epoch 100/100
171312/171312 [==============================] - ETA: 0s - loss: 1.4412 - acc: 0.696 - 208s 1ms/step - loss: 1.4412 - acc: 0.6962


## **Generating a sample shakespeare play**

In [37]:
seed_text = "Brevity is the soul of wit"
# Setting the length of the play
num_words = 20

In [38]:
# Predicting the next word from the seed text
for _ in range(num_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_seq_len-1, padding = "pre")
    predicted = model.predict_classes(token_list, verbose = 0)
    output = ""
    for word, idx in tokenizer.word_index.items():
        if idx == predicted:
            output = word
            break
    seed_text += " " + output

In [39]:
print(seed_text)

Brevity is the soul of wit bid it come not better it but first burthenous some sprays crooked horseman's gown talked month title title month message
